<div style="background-color:#000;"><img src="pqn.png"></img></div>

## Imports and setup

We bring in two trusted libraries here: one for pulling live financial market data, and another for organizing and working with that data in flexible tables.

In [ ]:
import yfinance as yf
import pandas as pd

These libraries let us tap into historical or real-time stock market info, as well as work with it seamlessly in ways that make analysis smooth and powerful. We use finance-specific tools from the first library and lean on the second for reliable data wrangling. There’s nothing fancy happening behind the scenes—just straightforward access to data and ways to shape it as needed.

## Retrieve historical stock prices

We're pulling the last year of price data for Apple and Microsoft so we can analyze how these companies' stocks have moved recently.

In [ ]:
apple = yf.Ticker("AAPL")
microsoft = yf.Ticker("MSFT")

In [ ]:
apple_price_data = apple.history(period="1y")
microsoft_price_data = microsoft.history(period="1y")

This code looks up each company by its ticker. Then, it fetches a full year’s worth of price data—like opening, closing, highs, and lows—so we have a clear view of these stocks’ behavior over time. Creating these pulls ahead of analysis saves us time and lets us explore trends, compare returns, or dig into specific dates quickly.

## Collect options data for Apple

Now, we’re grabbing Apple’s available options expiration dates and loading the calls and puts for the nearest expiry.

In [ ]:
apple_option_dates = apple.options
apple_option_chain = apple.option_chain(apple_option_dates[0])
apple_calls = apple_option_chain.calls
apple_puts = apple_option_chain.puts

The code first finds all the future dates when Apple options expire. It locks in the earliest one, then fetches two key sets of contracts—those for betting on the stock going up (calls) and those for betting on it going down (puts). With this live options data, we’re ready to dig deeper into options trading or pricing for Apple.

## Gather company financial statements

We’re pulling in the latest income statements, balance sheets, and cash flow tables for both Apple and Microsoft so we can look at the fundamentals.

In [ ]:
apple_financials = apple.financials
apple_balance_sheet = apple.balance_sheet
apple_cash_flow = apple.cashflow

In [ ]:
microsoft_financials = microsoft.financials
microsoft_balance_sheet = microsoft.balance_sheet
microsoft_cash_flow = microsoft.cashflow

Each of these steps reaches out and downloads the key financial documents that investors and analysts rely on most: how much money the companies made, what they own and owe, and how their cash moves in and out. We now have side-by-side financial snapshots of these giants, letting us compare them or study trends over time as we see fit.

## Pull recent prices and financials for tech leaders

We're batching price, options, and core financials for a group of major tech companies—Apple, Microsoft, Google, and Meta—for quick comparison or portfolio-wide analysis.

In [ ]:
symbols = ["AAPL", "MSFT", "GOOG", "META"]
price_data = {symbol: yf.Ticker(symbol).history(period="1mo") for symbol in symbols}

In [ ]:
options_exp_dates = {symbol: yf.Ticker(symbol).options for symbol in symbols}

In [ ]:
financials_data = {
    symbol: {
        "financials": yf.Ticker(symbol).financials,
        "balance_sheet": yf.Ticker(symbol).balance_sheet,
        "cash_flow": yf.Ticker(symbol).cashflow
    }
    for symbol in symbols
}

In [ ]:
financials_data

For each company in our group, we grab one month’s worth of price history, all of their available options expiries, and three detailed financial reports. This creates a tidy, easy-to-access set of data—one we can scan, visualize, or use in custom analysis. With this approach, any trends or hidden strengths become much easier to spot, whether we're trading or just tracking these industry leaders.

<a href="https://pyquantnews.com/">PyQuant News</a> is where finance practitioners level up with Python for quant finance, algorithmic trading, and market data analysis. Looking to get started? Check out the fastest growing, top-selling course to <a href="https://gettingstartedwithpythonforquantfinance.com/">get started with Python for quant finance</a>. For educational purposes. Not investment advice. Use at your own risk.